# 2.9.1 数组

## 1. 所有的 typecode 类型
<img src="./img/arr_typecode.png">

## 2-20 一个浮点数数组的创建，存入文件和文件读取的过程
> Python 数 C 语言数的数组类型很像，创建的时候都需要一个类型码，用来表示底层C语言的类型<br>
> 所以 array() 第一个参数就是用来表示数组的类型，我们可以看上面的表来确定

In [1]:
from array import array
from random import random

# 创建了一个有1000万个书浮点数随机数的数组
floats = array('d', (random() for i in range(10 ** 7)))
floats[-1]

0.4423777554229946

### 我们把生成的这个数组存入文件中

In [2]:
with open("./data/floats.bin", "wb") as f:
    # 写入 array 到一个文件流中
    floats.tofile(f)
print("Wrote floats to file")

Wrote floats to file


### 再读取这个文件中的数据

In [3]:
floats2 = array('d')  # create a double type array
with open("./data/floats.bin", "rb") as f:
    floats2.fromfile(f, 10 ** 7)  # 从流中读取数据到数组中

print(floats[-1])
print("Read floats from file DONE")

0.4423777554229946
Read floats from file DONE


> 验证下两个数组是否相同

In [4]:
print(floats == floats2)

True


## 2. 由于取消了 sort() 方法，我们要实现对其排序就得新建一个使用 sorted() 函数实现操作

In [16]:
demo = array('i', (5, 2, 3, 1, 4))
demo2 = array('i', sorted(demo))  # 没有 array.sort() 这个方法

demo2.tolist()  # 列表形式查看数组

[1, 2, 3, 4, 5]

---
# 2.9.2 内存视图
> memoryview() 就是对一个存储这个数组对象的内存里的字节进行操作，而不是对一个数本身操作<br>
> 所以它不需要拷贝对象，效率高，而且他还确保了安全

In [45]:
numbers = array('h', [-2, -1, 0, 1, 2])  # short 类型

memv = memoryview(numbers)  # 这里不是拷贝了一新对象，而是对老对象本身
print(f'数组的长度是：{len(memv):^5}')
print(f"第一个元素是：{memv[0]:^5}")

数组的长度是：  5  
第一个元素是： -2  


## memoryview.cast( ) 转换类型

In [46]:
memv_oct = memv.cast('B')  # 转换数组成 unsigned char 类型
memv_oct.tolist()  # 一列表形式查看数组内容

[254, 255, 255, 255, 0, 0, 1, 0, 2, 0]

> 当声明为 short 的数据转换成二进制后：
---
| 十进制 |             二进制              |
|-------|:----------------------------:|
| -2    |     0111 1111 1111 1111      |
| -1    |     1111 1111 1111 1111      |
| 0     |     0000 0000 0000 0000      |
| 1     |     1000 0000 0000 0000      |
| 2     |     0100 0000 0000 0000      |

> 我们从 unsigned short 转换成 unsigned char，是从 16 位变成 8 位，所以原本的一个 16 位表示一个会被拆成两个 8 位的数字<br>
> 于是就有了下面这个拆分后的表格:

---

| 十进制 |       二进制        |
|-------|:----------------:|
| 255   |    1111 1111     |
| -1    |    1111 1111     |
| 255   |    1111 1111     |
| 255   |    1111 1111     |
| 0     |    0000 0000     |
| 0     |    0000 0000     |
| 1     |    1000 0000     |
| 0     |    0000 0000     |
| 2     |    0100 0000     |
| 0     |    0000 0000     |


In [47]:
memv_oct[5] = 4  # 修改 第5个元素 修改成 4（0010 0000），这个不从 0 位置开始
numbers

array('h', [-2, -1, 1024, 1, 2])

> 我们的 memv_oct 是 [254, 255, 255, 255, 0, 0, 1, 0, 2, 0]
> 就变成了 [255, 255, 255, 255, 0, 4, 1, 0, 2, 0]
---
| 十进制 |    二进制    |
|-----|:---------:|
| 255 | 1111 1111 |
| -1  | 1111 1111 |
| 255 | 1111 1111 |
| 255 | 1111 1111 |
| 4   | 0000 0100 |
| 0   | 0000 0000 |
| 1   | 1000 0000 |
| 0   | 0000 0000 |
| 2   | 0100 0000 |
| 0   | 0000 0000 |
> 由于我们最后还要组合在一起，所以两个 8 位的unsigned char，还得组成一个 16 位的 unsigned short
> 所以新的数据就是
---
| 十进制  |         二进制          |
|------|:--------------------:|
| -2   | 0111 1111 1111 1111  |
| -1   | 1111 1111 1111 1111  |
| 1024 | 0000 0100 0000 0000  |
| 1    | 1000 0000 0000 0000  |
| 2    | 0100 0000 0000 0000  |

# 2.9.3 Numpy 和 Scipy

## 1. 创建一个 numpy 数组

In [26]:
import numpy as np

a = np.arange(12)  # 新建一个 0~11 的整数数组

a, type(a)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), numpy.ndarray)

## 2. 查看数组的维度

In [27]:
a.shape

(12,)

> 元素的长度决定了数组的维度

## 3. 转换数组的维度

In [30]:
a.shape = 3, 4
print(a)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


## 4. 多维数组的切片操作

In [32]:
print(a[2])

[ 8  9 10 11]


### 4.1 每一行的第二列元素

In [52]:
print(a[:,1])

[1 5 9]


### 4.2 第二行第二个元素

In [55]:
print(a[1, 1])

5


### 4.3 转置

In [59]:
print(a)

# 两种方法都可以
print(a.transpose())
print(a.T)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]
[[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]


## 5. numpy 对于文件的读写操作

### 5.1 读写文本文件

In [71]:
import numpy as np

# 生成一个 numpy 可读取的小数数组，存储在文本文件当中
data = np.random.rand(30,30)    # 30行30列，存储的随机是随机的浮点数

np.savetxt("./data/data30x30.txt", data)

print("File archived")

File archived


In [77]:
# 从文本文件中读取数据
data = np.loadtxt('./data/data30x30.txt', encoding='latin1')

# 打印数据
# print(data)
print(data[:,0], f"\n列 = {len(data[:, 0])}")

[0.23425691 0.5718938  0.49428931 0.04181783 0.88107904 0.26504135
 0.38728521 0.33860819 0.1747045  0.60098587 0.35929225 0.7813373
 0.04165275 0.30115213 0.41101212 0.94884771 0.80857693 0.62353858
 0.08662706 0.41856872 0.12801693 0.05012924 0.81392048 0.07597935
 0.63746285 0.64135026 0.38110165 0.94646598 0.71411066 0.01123031] 
列 = 30


### 5.2 读写二进制文件

In [78]:
# 把文本文件存储成二进制文件
data = np.random.rand(30,30)    # 30行30列，存储的随机的浮点数

np.save("./data/new_data30x30", data)

print("File archived")

File archived


In [79]:
# 从二进制文件中读取数据
data = np.load('./data/new_data30x30.npy')

# 打印数据
print(data)

[[0.75090901 0.21678775 0.56490658 0.48815209 0.4516253  0.01438757
  0.40533658 0.21447449 0.25390332 0.45000832 0.03618782 0.29472507
  0.83415263 0.81159348 0.33671359 0.48093474 0.77604094 0.35745186
  0.83924945 0.76346145 0.8956036  0.11068587 0.18917433 0.64996193
  0.89973748 0.65278707 0.63666325 0.02384529 0.43368843 0.63630694]
 [0.15919161 0.00525802 0.08052909 0.17894506 0.21823575 0.91186335
  0.63072415 0.20589622 0.36228693 0.98895669 0.96286837 0.8191429
  0.61503553 0.02090334 0.4878062  0.25380922 0.39058733 0.48856465
  0.49592524 0.88582761 0.83450497 0.17175497 0.56491859 0.96248478
  0.19352826 0.79166113 0.94348024 0.20790498 0.11789206 0.03601759]
 [0.75161272 0.01120239 0.68350395 0.19102345 0.62897566 0.99703291
  0.58807719 0.47118806 0.04677935 0.09737008 0.96847734 0.36753888
  0.32824851 0.89574658 0.37509244 0.12936337 0.21507712 0.32975592
  0.41625711 0.63541674 0.45413712 0.39454405 0.31219861 0.4282418
  0.70935664 0.30671984 0.41625972 0.53758777 0.

# 2.9.4 双向队列和其他形式的队列

## 2-23 使用双向队列

In [82]:
from collections import deque

dq = deque(range(10), maxlen=10)

dq

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)

### 1、队列的旋转操作
> routate 接受一个参数 n <br>
> 当 n 大于 0 时，将把队列最右边的n个元素移动到队列的最左边
> 当 n 小于 0 时，将把队列最右边的n个元素移动到队列的最右边

In [83]:
dq.rotate(3)
dq

deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6], maxlen=10)

> 左边三个移动到右边

In [84]:
dq.rotate(-4)
dq

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0], maxlen=10)

### 2、队列满的时候添加新数据的处理
> 如果一个队列已满，那么添加头就会删除后面的，腾出空间。
> 相反，如果后面添加元素，那么前面会删掉，腾出空间

In [86]:
# 前面塞一个元素
dq.appendleft(-1)
dq  # 删除了尾巴的元素

deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)

In [87]:
# 后面添加三个元素
dq.extend([1,2,3])
dq  #  删除了前面的元素 -1，1，2

deque([3, 4, 5, 6, 7, 8, 9, 1, 2, 3], maxlen=10)

In [88]:
# 前面添加几个元素
dq.extendleft([-1, -2, -3])
dq    #  删除了后面的元素 1, 2, 3

deque([-3, -2, -1, 3, 4, 5, 6, 7, 8, 9], maxlen=10)

### 3、查看元素是否存在于列表中

In [89]:
from collections import deque

# 创建一个deque
my_deque = deque([1, 2, 3, 4, 5])

# 判断元素是否存在于deque中
if 3 in my_deque:
    print("元素存在于deque中")
else:
    print("元素不存在于deque中")

元素存在于deque中
